# List of My Movies

In [10]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from combine import combine
from fsUtils import moveFile


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-03-02 11:14:56.054319


In [11]:
import re
from time import sleep
from collections import OrderedDict
from timeUtils import clock, elapsed
from ioUtils import saveFile, getFile
from fsUtils import setDir, isDir, mkDir, setFile, isFile, setSubFile, moveFile
from fileUtils import getBaseFilename, getDirname, getFileBasics
from searchUtils import findSubPatternExt, findPatternExt, findExt, findNearest
from strUtils import convertCurrency
from webUtils import getWebData, getHTML
from numpy import repeat
from movieDB import movieDB
from os import getcwd
from os.path import join
from glob import glob
import operator
from movieRenames import manualRenames

##############################################################################################################################
# Combine Movies
##############################################################################################################################
class mymovies(movieDB):
    def __init__(self, basedir=None):
        self.name = "mymovies"
        movieDB.__init__(self, dbdir=self.name)
        self.combine = None
        
    def setCombinedMovies(self, combine):
        self.combine = combine
        
    def getCombinedMovies(self, debug=False):        
        savename = setFile(self.combine.getResultsDir(), "movies.json")
        if not isFile(savename):
            raise ValueErrro("Cannot access {0}".format(savename))
        combinedMovies = getFile(savename)
        if debug:
            print("Found {0} combined movies".format(len(combinedMovies)))
        return combinedMovies
    
    def setMissingMovies(self, missing):
        self.missingMovies = missing
        
    def getMissingMovies(self):
        return self.missingMovies
        
    def getMyMovies(self, debug=False): 
        savename = setFile(self.getDataDir(), "mymovies.json")
        if not isFile(savename):
            raise ValueErrro("Cannot access {0}".format(savename))
        mine = getFile(savename)
        if debug:
            print("Found {0} my movies".format(len(mine)))
        return mine
        
    def findMyMovies(self, debug=False):
        movies = glob("/Volumes/*/Movies/*")
        mine   = dict(zip([getBaseFilename(x) for x in movies], movies))
        print("Found {0} movies on my disks".format(len(movies)))
        savename = setFile(self.getDataDir(), "mymovies.json")
        saveFile(idata=mine, ifile=savename, debug=True)
        
        
    def searchMyMovies(self, movie, year, cutoff=0.8):
        moviesMine = self.getMyMovies()
        key = "{0} [{1}]".format(movie, year)
        result     = findNearest(key, moviesMine.keys(), 3, 0.95)
        if len(result) > 0:
            print("{0}  --->  {1}".format(movie, result))
            location = moviesMine[result[0]]
            print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)\n".format(location, location))
            return
        else:
            key = "{0} [{1}]".format(movie, year)
            result = findNearest(key, moviesMine.keys(), 3, cutoff)
            if len(result) > 0:
                print("{0}  --->  {1}".format(movie, result))
                location = moviesMine[result[0]]
                print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)\n".format(location, location))
                return
        print("No match found")
        return
    
    
    def suggestRenames(self):

        movies = glob("/Volumes/Download/MoviesFinished/*.*")
        for movie in movies:
            fileInfo = getFileBasics(movie)
            name = fileInfo[1]
                        
            ps=["1080", "720", "480"]
            for p in ps:
                name = name.replace("{0}p".format(p), "")

            xs=['264']
            for x in xs:
                name = name.replace("x{0}".format(x), "")
            
            ts=["BluRay", "BRrip", "WEBRip", "HDTVRip", "GAZ", "BrRip"]
            for t in ts:
                name = name.replace("{0}".format(t), "")
                
            vs=["-[YTS.AG]", "-[YTS.AM]", "YIFY", "[YTS.AG]"]
            for v in vs:
                name = name.replace("{0}".format(v), "")
                
            name = name.replace(".", " ")
            name=name.strip()

            dst = join(fileInfo[0], "".join([name, fileInfo[2]]))
            if name != fileInfo[1]:
                moveFile(src=movie, dst=dst, debug=True)


        movies = glob("/Volumes/Download/MoviesFinished/*.*")   
        for movie in movies:
            fileInfo = getFileBasics(movie)            
            name  = fileInfo[1][:-4].strip()
            year  = fileInfo[1][-4:]
            
            try:
                int(year)
                name = "{0} [{1}]".format(name, year)
            except:
                print("Cannot rename {0}".format(movie))
                continue
            
            src = movie
            dst = join(fileInfo[0], "".join([name, fileInfo[2]]))
            moveFile(src=src, dst=dst, debug=True)
            print("")
        
        
    def mergeMovies(self, toFile=False, movie=None, source=None, year=None):
        moviesCombined = self.getCombinedMovies()
        moviesMine     = self.getMyMovies()
        renames        = {}
        filename = setFile(self.getResultsDir(), "mymovies.dat")
        
        
        yearcut = year
        sourcecut = source
        moviecut = movie

        if toFile:
            print("{0: <6}{1: <3}{2: <100}{3: <20}".format("Year", u'\u2713', "Movie", "Source"), file=open(filename, "w"))
        else:
            print("{0: <6}{1: <3}{2: <100}{3: <20}".format("Year", u'\u2713', "Movie", "Source"))

        pYear = None
        yearRenames = {}
        
        
        toGet = {}
        
        for key,name in moviesCombined.items():
            movie   = key[:-7]
            year = key[-5:-1]
            
            if yearcut is not None:
                if int(year) not in [int(yearcut)-1,int(yearcut),int(yearcut)+1]:
                    continue
            if sourcecut is not None:
                if name != sourcecut:
                    continue
            if moviecut is not None:
                if moviecut not in movie:
                    continue

        
            location = moviesMine.get(key)
            if location is None:
                for dYear in [-1,1,-2,2]:
                    dKey = "{0} [{1}]".format(movie, int(year)+dYear)
                    if moviesMine.get(dKey) is not None:
                        location = moviesMine[dKey]
                        break
            if location is None:
                result = findNearest(key, moviesMine.keys(), 1, 0.98)

                if len(result) > 0:
                    location = u'\u2248'
                    myMovie  = moviesMine[result[0]]
                    fileInfo = getFileBasics(myMovie)
                    myMovieYear = fileInfo[1][-5:-1]

                    yearRenames[myMovie] = join(fileInfo[0], "".join([key, fileInfo[2]]))

                    if abs(int(year) - int(myMovieYear)) <= 0:
                        renames[myMovie] = join(fileInfo[0], "".join([key, fileInfo[2]]))
                else:
                    result = findNearest(key, moviesMine.keys(), 1, 0.90)
                    if len(result) > 0:
                        location = u'\u2047'
                        myMovie  = moviesMine[result[0]]
                        fileInfo = getFileBasics(myMovie)
                        myMovieYear = fileInfo[1][-5:-1]

                        yearRenames[myMovie] = join(fileInfo[0], "".join([key, fileInfo[2]]))

                        if abs(int(year) - int(myMovieYear)) <= 0:
                            renames[myMovie] = join(fileInfo[0], "".join([key, fileInfo[2]]))
                    else:
                        location = ""
                        
                        if toGet.get(year) is None:
                            toGet[year] = set()
                        toGet[year].add(movie)
            else:
                location = u'\u2713'
                
            
            #print(pYear,year)
            if pYear is not None and pYear != year:
                print("\n")
                for key,rename in yearRenames.items():
                    if key != rename:
                        print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)".format(key, rename))                
                
                print("\n")
                yearRenames = {}
            pYear = year
                
            if toFile:
                print("{0: <6}{1: <3}{2: <100}{3: <20}".format(year, location, movie, name), file=open(filename, "a"))
            else:
                print("{0: <6}{1: <3}{2: <100}{3: <20}".format(year, location, movie, name))

                
                
        print("\n")
        for key,rename in renames.items():
            if key != rename:
                print("moveFile(src=\"{0}\", dst=\"{1}\", debug=True)\n".format(key, rename))

        self.setMissingMovies(toGet)
        print("Missing Movies Found in getMissingMovies()")

In [12]:
mine = mymovies()
mine.setCombinedMovies(combine())
_ = clock("Last Run")

Current Time is Sat Mar 02, 2019 11:15:10 for Last Run


In [14]:
mine.suggestRenames()

In [15]:
mine.findMyMovies()
_ = clock("Last Run")

Found 5624 movies on my disks
Saving data to /Users/tgadfort/Documents/code/movies/mymovies/data/mymovies.json
Saved data to /Users/tgadfort/Documents/code/movies/mymovies/data/mymovies.json
  --> This file is 455.4kB.
Current Time is Sat Mar 02, 2019 11:16:30 for Last Run


In [19]:
mine.searchMyMovies("Free Solo", 2018, cutoff=0.6)

Free Solo  --->  ['Free Solo [2018]']
moveFile(src="/Volumes/Download/Movies/Free Solo [2018].mp4", dst="/Volumes/Download/Movies/Free Solo [2018].mp4", debug=True)



In [ ]:
### Stopped after 1985. Do 1986 onward

mine.mergeMovies()

Year  ✓  Movie                                                                                               Source              
1900     Faust And Marguerite                                                                                Films101            
1900     Grandma's Reading Glass                                                                             Films101            
1900  ✓  Jeanne d'Arc                                                                                        Films101            
1900     Le Rêve de Noël                                                                                     Films101            
1900     The Mystic Swing                                                                                    Films101            
1900     Why Mrs. Jones Got A Divorce                                                                        Films101            
1900  ⁇  Cyrano De Bergerac                                                               

1907     Hamlet                                                                                              Films101            
1907     The Haunted Hotel                                                                                   Films101            
1907     The Magic Fountain Pen                                                                              Films101            
1907     The Midnight Ride Of Paul Revere                                                                    Films101            
1907     Salaviinanpolttajat                                                                                 Films101            
1907     The Red Spectre                                                                                     Films101            




1908     The Adventures Of Dollie                                                                            Films101            
1908     The Fatal Hour                                                               

1911  ⁇  Romeo And Juliet                                                                                    Films101            
1911     A Tale Of Two Cities                                                                                Films101            
1911     Vanity Fair                                                                                         Films101            
1911     Little Nemo                                                                                         Films101            
1911     Billy And His Pal                                                                                   Films101            
1911     Manhattan Trade School For Girls                                                                    Films101            
1911     A Western Girl                                                                                      Films101            


moveFile(src="/Volumes/Save/Movies/Romeo And Juliet [1968].mp4", dst="/Volumes/Save/Movi

1914     A Child Of The Big City                                                                             Films101            
1914     Kid Auto Races At Venice                                                                            Films101            
1914     Making A Living                                                                                     Films101            
1914     The Wishing Ring: An Idyll Of Old England                                                           Films101            
1914     Dough And Dynamite                                                                                  Films101            
1914     In The Sage Brush Country                                                                           Films101            
1914     Judith Of Bethulia                                                                                  Films101            
1914     Lieutenant Pimple And The Stolen Invention                                       

1916     The Vagabond                                                                                        Films101            
1916     Shoes                                                                                               Films101            
1916     Arsene Lupin                                                                                        Films101            
1916     The Cossack Whip                                                                                    Films101            
1916     A Daughter Of The Gods                                                                              Films101            
1916     His Picture In The Papers                                                                           Films101            
1916     Hoodoo Ann                                                                                          Films101            
1916     Joan The Woman                                                                   

...

1994     Backbeat                                                                                            BAFTA               
1994     Bhaji On The Beach                                                                                  BAFTA               
1994     Shallow Grave                                                                                       BAFTA               
1994     Priest                                                                                              BAFTA               
1994  ≈  The Adventures Of Priscilla: Queen Of The Desert                                                    Filmsite            
1994     Chungking Express                                                                                   Filmsite            
1994  ⁇  Clerks.                                                                                             Filmsite            
1994  ✓  Dumb & Dumber                                                                    

1994     Imaginary Crimes                                                                                    Films101            
1994     Immortal Beloved                                                                                    Films101            
1994     I.Q.                                                                                                Films101            
1994     Junior                                                                                              Films101            
1994     Little Big League                                                                                   Films101            
1994     Only You                                                                                            Films101            
1994     The Paper                                                                                           Films101            
1994     Sirens                                                                           

1994     A Low Down Dirty Shame                                                                              Box Office Mojo     
1994     I Love Trouble                                                                                      Box Office Mojo     
1994  ✓  Blown Away                                                                                          Box Office Mojo     
1994     In The Army Now                                                                                     Box Office Mojo     
1994     Drop Zone                                                                                           Box Office Mojo     
1994  ✓  The Jungle Book                                                                                     Box Office Mojo     
1994     My Father, The Hero                                                                                 Box Office Mojo     
1994  ✓  Street Fighter                                                                   

1995     One Survivor Remembers                                                                              Oscar               
1995     Antonia's Line                                                                                      Oscar               
1995     Lieberman In Love                                                                                   Oscar               
1995  ✓  Pocahontas                                                                                          Oscar               
1995  ✓  Before Sunrise                                                                                      Rotten Tomatoes     
1995  ✓  The Secret Of Roan Inish                                                                            Rotten Tomatoes     
1995  ✓  Crumb                                                                                               Rotten Tomatoes     
1995  ⁇  Richard III                                                                      

1995     A Midwinter's Tale                                                                                  Films101            
1995     Once Upon A Time . . . When We Were Colored                                                         Films101            
1995     Dust Of Life                                                                                        Films101            
1995     O Quatrilho                                                                                         Films101            
1995     Scratch And Crow                                                                                    Films101            
1995     The Celluloid Closet                                                                                Films101            
1995     The Brothers McMullen                                                                               Films101            
1995  ✓  Copycat                                                                          

In [ ]:

missingMovies = mine.getMissingMovies()
print("====================== Missing Movies ======================")
print("{0: <10}{1: <100}".format("Year", "Movie"))
print("{0: <10}{1: <100}".format("----", "-----"))

for year in missingMovies.keys():
    print("")
    for movie in sorted(missingMovies[year]):
        print("{0: <10}{1: <100}".format(year, movie))
        continue
        
        pos = movie.rfind("(")
        if pos == -1:
            continue        
        cpos = movie.rfind(",")
        if cpos == -1:
            continue
        fix = movie[:pos].strip()
        print("{0}:".format(fix))
        print(" - {0}".format(movie))
        #print("")



In [52]:

moveFile(src="/Volumes/Download/Movies/The Thing From Another World [1951].mp4", dst="/Volumes/Download/Movies/The Thing (From Another World) [1951].mp4", debug=True)

Moving /Volumes/Download/Movies/The Thing From Another World [1951].mp4 to /Volumes/Download/Movies/The Thing (From Another World) [1951].mp4


In [50]:

moveFile(src="/Volumes/Seagate/Movies/Sunday Bloody Sunday [1971].mp4", dst="/Volumes/Seagate/Movies/Sunday, Bloody Sunday [1971].mp4", debug=True)


Moving /Volumes/Seagate/Movies/Sunday Bloody Sunday [1971].mp4 to /Volumes/Seagate/Movies/Sunday, Bloody Sunday [1971].mp4
